### Preprocessing the Data for a Neural Network

In [36]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [37]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# Optimization attempt drop 'APPLICATION_TYPE'
application_df=application_df.drop(columns=["EIN","NAME","APPLICATION_TYPE"])

In [38]:
# Determine the number of unique values in each column.
application_df.nunique()

AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [39]:
# Look at CLASSIFICATION value counts for binning
class_count=application_df.CLASSIFICATION.value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1236        1
C6100        1
C3700        1
C2170        1
C1248        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [40]:
# Determine which values to replace if counts are less than ..?
replace_class=list(class_count[class_count<1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [41]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [42]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df=pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [43]:
# Merge one-hot encoded features and drop the originals
application_df=application_df.merge(encode_df,left_index=True,right_index=True)
application_df=application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [44]:
# Split our preprocessed data into our features and target arrays
y=application_df["IS_SUCCESSFUL"].values
X=application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [45]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate the Model

In [46]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense
       (units=hidden_nodes_layer1, input_dim=number_input_features, 
                          activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 80)                2800      
_________________________________________________________________
dense_11 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 31        
Total params: 5,261
Trainable params: 5,261
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [48]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [49]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 91/804 [==>...........................] - ETA: 0s - loss: 0.7067 - accuracy: 0.5996
Epoch 00001: saving model to checkpoints\weights.01.hdf5
192/804 [======>.......................] - ETA: 2s - loss: 0.6642 - accuracy: 0.6404
Epoch 00001: saving model to checkpoints\weights.01.hdf5
274/804 [=========>....................] - ETA: 2s - loss: 0.6487 - accuracy: 0.6547
Epoch 00001: saving model to checkpoints\weights.01.hdf5
400/804 [=============>................] - ETA: 1s - loss: 0.6346 - accuracy: 0.6672
Epoch 00001: saving model to checkpoints\weights.01.hdf5
470/804 [================>.............] - ETA: 1s - loss: 0.6293 - accuracy: 0.6717
Epoch 00001: saving model to checkpoints\weights.01.hdf5
600/804 [=====================>........] - ETA: 0s - loss: 0.6229 - accuracy: 0.6769
Epoch 00001: saving model to checkpoints\weights.01.hdf5
686/804 [========================>.....] - ETA: 0s - loss: 0.6200 - accuracy: 0.6793
Epoch 00001: saving model to checkpoints\weights.01

563/804 [====================>.........] - ETA: 0s - loss: 0.5793 - accuracy: 0.7056
Epoch 00007: saving model to checkpoints\weights.07.hdf5
652/804 [=======================>......] - ETA: 0s - loss: 0.5788 - accuracy: 0.7061
Epoch 00007: saving model to checkpoints\weights.07.hdf5
776/804 [===========================>..] - ETA: 0s - loss: 0.5783 - accuracy: 0.7068
Epoch 00007: saving model to checkpoints\weights.07.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5782 - accuracy: 0.7069
Epoch 8/100
 40/804 [>.............................] - ETA: 0s - loss: 0.5622 - accuracy: 0.7264
Epoch 00008: saving model to checkpoints\weights.08.hdf5
159/804 [====>.........................] - ETA: 1s - loss: 0.5687 - accuracy: 0.7212
Epoch 00008: saving model to checkpoints\weights.08.hdf5
269/804 [=========>....................] - ETA: 1s - loss: 0.5710 - accuracy: 0.7180
Epoch 00008: saving model to checkpoints\weights.08.hdf5
331/804 [===========>..................] - ETA:

209/804 [======>.......................] - ETA: 1s - loss: 0.5827 - accuracy: 0.6997
Epoch 00014: saving model to checkpoints\weights.14.hdf5
320/804 [==========>...................] - ETA: 1s - loss: 0.5810 - accuracy: 0.7031
Epoch 00014: saving model to checkpoints\weights.14.hdf5
442/804 [===============>..............] - ETA: 0s - loss: 0.5791 - accuracy: 0.7060
Epoch 00014: saving model to checkpoints\weights.14.hdf5
531/804 [==================>...........] - ETA: 0s - loss: 0.5782 - accuracy: 0.7073
Epoch 00014: saving model to checkpoints\weights.14.hdf5
612/804 [=====================>........] - ETA: 0s - loss: 0.5777 - accuracy: 0.7081
Epoch 00014: saving model to checkpoints\weights.14.hdf5
719/804 [=========================>....] - ETA: 0s - loss: 0.5771 - accuracy: 0.7089
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5766 - accuracy: 0.7095
Epoch 15/100
 43/804 [>.............................] - ETA

686/804 [========================>.....] - ETA: 0s - loss: 0.5684 - accuracy: 0.7194
Epoch 00020: saving model to checkpoints\weights.20.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5688 - accuracy: 0.7187
Epoch 21/100
  1/804 [..............................] - ETA: 1s - loss: 0.4865 - accuracy: 0.7812
Epoch 00021: saving model to checkpoints\weights.21.hdf5
105/804 [==>...........................] - ETA: 3s - loss: 0.5778 - accuracy: 0.6989
Epoch 00021: saving model to checkpoints\weights.21.hdf5
215/804 [=======>......................] - ETA: 2s - loss: 0.5805 - accuracy: 0.7005
Epoch 00021: saving model to checkpoints\weights.21.hdf5
297/804 [==========>...................] - ETA: 1s - loss: 0.5801 - accuracy: 0.7018
Epoch 00021: saving model to checkpoints\weights.21.hdf5
420/804 [==============>...............] - ETA: 1s - loss: 0.5790 - accuracy: 0.7038 ETA: 1s - loss: 0.5793 - accuracy
Epoch 00021: saving model to checkpoints\weights.21.hdf5
503/804 [===

289/804 [=========>....................] - ETA: 1s - loss: 0.5717 - accuracy: 0.7168
Epoch 00027: saving model to checkpoints\weights.27.hdf5
395/804 [=============>................] - ETA: 1s - loss: 0.5717 - accuracy: 0.7159
Epoch 00027: saving model to checkpoints\weights.27.hdf5
471/804 [================>.............] - ETA: 1s - loss: 0.5713 - accuracy: 0.7159
Epoch 00027: saving model to checkpoints\weights.27.hdf5
575/804 [====================>.........] - ETA: 0s - loss: 0.5710 - accuracy: 0.7159
Epoch 00027: saving model to checkpoints\weights.27.hdf5
658/804 [=======================>......] - ETA: 0s - loss: 0.5708 - accuracy: 0.7157
Epoch 00027: saving model to checkpoints\weights.27.hdf5
782/804 [============================>.] - ETA: 0s - loss: 0.5707 - accuracy: 0.7155
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5707 - accuracy: 0.7155
Epoch 28/100
 86/804 [==>...........................] - ETA

...

174/804 [=====>........................] - ETA: 1s - loss: 0.5680 - accuracy: 0.7134
Epoch 00073: saving model to checkpoints\weights.73.hdf5
275/804 [=========>....................] - ETA: 1s - loss: 0.5668 - accuracy: 0.7150
Epoch 00073: saving model to checkpoints\weights.73.hdf5
386/804 [=============>................] - ETA: 1s - loss: 0.5670 - accuracy: 0.7154
Epoch 00073: saving model to checkpoints\weights.73.hdf5
490/804 [=================>............] - ETA: 0s - loss: 0.5674 - accuracy: 0.7152
Epoch 00073: saving model to checkpoints\weights.73.hdf5
612/804 [=====================>........] - ETA: 0s - loss: 0.5680 - accuracy: 0.7150
Epoch 00073: saving model to checkpoints\weights.73.hdf5
701/804 [=========================>....] - ETA: 0s - loss: 0.5682 - accuracy: 0.7150
Epoch 00073: saving model to checkpoints\weights.73.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5682 - accuracy: 0.7151
Epoch 74/100
  1/804 [..............................] - ETA

662/804 [=======================>......] - ETA: 0s - loss: 0.5643 - accuracy: 0.7169
Epoch 00079: saving model to checkpoints\weights.79.hdf5
777/804 [===========================>..] - ETA: 0s - loss: 0.5649 - accuracy: 0.7166
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5650 - accuracy: 0.7166
Epoch 80/100
 84/804 [==>...........................] - ETA: 1s - loss: 0.5688 - accuracy: 0.7052
Epoch 00080: saving model to checkpoints\weights.80.hdf5
154/804 [====>.........................] - ETA: 2s - loss: 0.5710 - accuracy: 0.7073
Epoch 00080: saving model to checkpoints\weights.80.hdf5
277/804 [=========>....................] - ETA: 1s - loss: 0.5697 - accuracy: 0.7104
Epoch 00080: saving model to checkpoints\weights.80.hdf5
367/804 [============>.................] - ETA: 1s - loss: 0.5691 - accuracy: 0.7119
Epoch 00080: saving model to checkpoints\weights.80.hdf5
480/804 [================>.............] - ETA

319/804 [==========>...................] - ETA: 1s - loss: 0.5570 - accuracy: 0.7247
Epoch 00086: saving model to checkpoints\weights.86.hdf5
426/804 [==============>...............] - ETA: 0s - loss: 0.5573 - accuracy: 0.7247
Epoch 00086: saving model to checkpoints\weights.86.hdf5
553/804 [===================>..........] - ETA: 0s - loss: 0.5585 - accuracy: 0.7241
Epoch 00086: saving model to checkpoints\weights.86.hdf5
638/804 [======================>.......] - ETA: 0s - loss: 0.5593 - accuracy: 0.7235
Epoch 00086: saving model to checkpoints\weights.86.hdf5
727/804 [==========================>...] - ETA: 0s - loss: 0.5600 - accuracy: 0.7230
Epoch 00086: saving model to checkpoints\weights.86.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5606 - accuracy: 0.7225
Epoch 87/100
 45/804 [>.............................] - ETA: 0s - loss: 0.5606 - accuracy: 0.7191
Epoch 00087: saving model to checkpoints\weights.87.hdf5
152/804 [====>.........................] - ETA

804/804 [==============================] - 2s 3ms/step - loss: 0.5699 - accuracy: 0.7124
Epoch 93/100
 30/804 [>.............................] - ETA: 1s - loss: 0.5693 - accuracy: 0.6983
Epoch 00093: saving model to checkpoints\weights.93.hdf5
 99/804 [==>...........................] - ETA: 3s - loss: 0.5792 - accuracy: 0.6960
Epoch 00093: saving model to checkpoints\weights.93.hdf5
194/804 [======>.......................] - ETA: 2s - loss: 0.5760 - accuracy: 0.7032
Epoch 00093: saving model to checkpoints\weights.93.hdf5
300/804 [==========>...................] - ETA: 1s - loss: 0.5735 - accuracy: 0.7073
Epoch 00093: saving model to checkpoints\weights.93.hdf5
427/804 [==============>...............] - ETA: 1s - loss: 0.5714 - accuracy: 0.7105
Epoch 00093: saving model to checkpoints\weights.93.hdf5
512/804 [==================>...........] - ETA: 0s - loss: 0.5705 - accuracy: 0.7118
Epoch 00093: saving model to checkpoints\weights.93.hdf5
623/804 [======================>.......] - ETA

482/804 [================>.............] - ETA: 1s - loss: 0.5639 - accuracy: 0.7197
Epoch 00099: saving model to checkpoints\weights.99.hdf5
606/804 [=====================>........] - ETA: 0s - loss: 0.5639 - accuracy: 0.7202
Epoch 00099: saving model to checkpoints\weights.99.hdf5
705/804 [=========================>....] - ETA: 0s - loss: 0.5642 - accuracy: 0.7201
Epoch 00099: saving model to checkpoints\weights.99.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5644 - accuracy: 0.7199
Epoch 100/100
  1/804 [..............................] - ETA: 2s - loss: 0.6133 - accuracy: 0.6250
Epoch 00100: saving model to checkpoints\weights.100.hdf5
104/804 [==>...........................] - ETA: 2s - loss: 0.5690 - accuracy: 0.7047
Epoch 00100: saving model to checkpoints\weights.100.hdf5
184/804 [=====>........................] - ETA: 2s - loss: 0.5680 - accuracy: 0.7084
Epoch 00100: saving model to checkpoints\weights.100.hdf5
262/804 [========>.....................] -

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5813 - accuracy: 0.7058
Loss: 0.5812931656837463, Accuracy: 0.7057725787162781


In [51]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

### Optimization Attempt with New Model

In [55]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn_attempt2 = tf.keras.models.Sequential()

# First hidden layer
nn_attempt2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_attempt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="softplus"))

# Output layer
nn_attempt2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_attempt2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_attempt2.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_attempt2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5832 - accuracy: 0.7036
Loss: 0.5831734538078308, Accuracy: 0.7035568356513977


In [56]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn_attempt3 = tf.keras.models.Sequential()

# First hidden layer
nn_attempt3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_attempt3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_attempt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_attempt3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_attempt3.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
 73/804 [=>............................] - ETA: 1s - loss: 0.6563 - accuracy: 0.6340
Epoch 00001: saving model to checkpoints\weights.01.hdf5
168/804 [=====>........................] - ETA: 1s - loss: 0.6310 - accuracy: 0.6633
Epoch 00001: saving model to checkpoints\weights.01.hdf5
300/804 [==========>...................] - ETA: 1s - loss: 0.6204 - accuracy: 0.6739
Epoch 00001: saving model to checkpoints\weights.01.hdf5
376/804 [=============>................] - ETA: 0s - loss: 0.6170 - accuracy: 0.6767
Epoch 00001: saving model to checkpoints\weights.01.hdf5
457/804 [================>.............] - ETA: 0s - loss: 0.6142 - accuracy: 0.6793
Epoch 00001: saving model to checkpoints\weights.01.hdf5
565/804 [====================>.........] - ETA: 0s - loss: 0.6113 - accuracy: 0.6820
Epoch 00001: saving model to checkpoints\weights.01.hdf5
666/804 [=======================>......] - ETA: 0s - loss: 0.6093 - accuracy: 0.6836
Epoch 00001: saving model to checkpoints\weights.01.

566/804 [====================>.........] - ETA: 0s - loss: 0.5744 - accuracy: 0.7127
Epoch 00007: saving model to checkpoints\weights.07.hdf5
671/804 [========================>.....] - ETA: 0s - loss: 0.5747 - accuracy: 0.7122
Epoch 00007: saving model to checkpoints\weights.07.hdf5
767/804 [===========================>..] - ETA: 0s - loss: 0.5748 - accuracy: 0.7120
Epoch 00007: saving model to checkpoints\weights.07.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5748 - accuracy: 0.7119
Epoch 8/50
 35/804 [>.............................] - ETA: 1s - loss: 0.5446 - accuracy: 0.7512
Epoch 00008: saving model to checkpoints\weights.08.hdf5
171/804 [=====>........................] - ETA: 1s - loss: 0.5641 - accuracy: 0.7249
Epoch 00008: saving model to checkpoints\weights.08.hdf5
253/804 [========>.....................] - ETA: 1s - loss: 0.5664 - accuracy: 0.7222
Epoch 00008: saving model to checkpoints\weights.08.hdf5
359/804 [============>.................] - ETA: 

221/804 [=======>......................] - ETA: 1s - loss: 0.5619 - accuracy: 0.7180
Epoch 00014: saving model to checkpoints\weights.14.hdf5
345/804 [===========>..................] - ETA: 1s - loss: 0.5654 - accuracy: 0.7166
Epoch 00014: saving model to checkpoints\weights.14.hdf5
412/804 [==============>...............] - ETA: 1s - loss: 0.5666 - accuracy: 0.7160
Epoch 00014: saving model to checkpoints\weights.14.hdf5
508/804 [=================>............] - ETA: 0s - loss: 0.5678 - accuracy: 0.7155
Epoch 00014: saving model to checkpoints\weights.14.hdf5
620/804 [======================>.......] - ETA: 0s - loss: 0.5685 - accuracy: 0.7155
Epoch 00014: saving model to checkpoints\weights.14.hdf5
743/804 [==========================>...] - ETA: 0s - loss: 0.5689 - accuracy: 0.7156
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5692 - accuracy: 0.7155
Epoch 15/50
 42/804 [>.............................] - ETA:

683/804 [========================>.....] - ETA: 0s - loss: 0.5759 - accuracy: 0.7117
Epoch 00020: saving model to checkpoints\weights.20.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5754 - accuracy: 0.7120
Epoch 21/50
  1/804 [..............................] - ETA: 1s - loss: 0.6626 - accuracy: 0.6562
Epoch 00021: saving model to checkpoints\weights.21.hdf5
114/804 [===>..........................] - ETA: 0s - loss: 0.5826 - accuracy: 0.7029
Epoch 00021: saving model to checkpoints\weights.21.hdf5
188/804 [======>.......................] - ETA: 1s - loss: 0.5790 - accuracy: 0.7054
Epoch 00021: saving model to checkpoints\weights.21.hdf5
302/804 [==========>...................] - ETA: 0s - loss: 0.5756 - accuracy: 0.7087
Epoch 00021: saving model to checkpoints\weights.21.hdf5
398/804 [=============>................] - ETA: 0s - loss: 0.5741 - accuracy: 0.7102
Epoch 00021: saving model to checkpoints\weights.21.hdf5
488/804 [=================>............] - ETA:

381/804 [=============>................] - ETA: 0s - loss: 0.5769 - accuracy: 0.7073
Epoch 00027: saving model to checkpoints\weights.27.hdf5
468/804 [================>.............] - ETA: 0s - loss: 0.5760 - accuracy: 0.7081
Epoch 00027: saving model to checkpoints\weights.27.hdf5
567/804 [====================>.........] - ETA: 0s - loss: 0.5751 - accuracy: 0.7091
Epoch 00027: saving model to checkpoints\weights.27.hdf5
673/804 [========================>.....] - ETA: 0s - loss: 0.5744 - accuracy: 0.7100
Epoch 00027: saving model to checkpoints\weights.27.hdf5
771/804 [===========================>..] - ETA: 0s - loss: 0.5739 - accuracy: 0.7108
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5738 - accuracy: 0.7110
Epoch 28/50
 87/804 [==>...........................] - ETA: 0s - loss: 0.5969 - accuracy: 0.6839
Epoch 00028: saving model to checkpoints\weights.28.hdf5
165/804 [=====>........................] - ETA:

804/804 [==============================] - 2s 3ms/step - loss: 0.5689 - accuracy: 0.7150
Epoch 34/50
 65/804 [=>............................] - ETA: 1s - loss: 0.5475 - accuracy: 0.7262
Epoch 00034: saving model to checkpoints\weights.34.hdf5
159/804 [====>.........................] - ETA: 1s - loss: 0.5553 - accuracy: 0.7220
Epoch 00034: saving model to checkpoints\weights.34.hdf5
262/804 [========>.....................] - ETA: 1s - loss: 0.5589 - accuracy: 0.7208
Epoch 00034: saving model to checkpoints\weights.34.hdf5
351/804 [============>.................] - ETA: 1s - loss: 0.5603 - accuracy: 0.7205
Epoch 00034: saving model to checkpoints\weights.34.hdf5
455/804 [===============>..............] - ETA: 0s - loss: 0.5619 - accuracy: 0.7198
Epoch 00034: saving model to checkpoints\weights.34.hdf5
553/804 [===================>..........] - ETA: 0s - loss: 0.5630 - accuracy: 0.7194
Epoch 00034: saving model to checkpoints\weights.34.hdf5
659/804 [=======================>......] - ETA:

525/804 [==================>...........] - ETA: 0s - loss: 0.5727 - accuracy: 0.7115
Epoch 00040: saving model to checkpoints\weights.40.hdf5
612/804 [=====================>........] - ETA: 0s - loss: 0.5722 - accuracy: 0.7119
Epoch 00040: saving model to checkpoints\weights.40.hdf5
715/804 [=========================>....] - ETA: 0s - loss: 0.5717 - accuracy: 0.7124
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5714 - accuracy: 0.7128
Epoch 41/50
 39/804 [>.............................] - ETA: 1s - loss: 0.5752 - accuracy: 0.7076
Epoch 00041: saving model to checkpoints\weights.41.hdf5
110/804 [===>..........................] - ETA: 1s - loss: 0.5727 - accuracy: 0.7141
Epoch 00041: saving model to checkpoints\weights.41.hdf5
220/804 [=======>......................] - ETA: 1s - loss: 0.5688 - accuracy: 0.7179
Epoch 00041: saving model to checkpoints\weights.41.hdf5
320/804 [==========>...................] - ETA:

183/804 [=====>........................] - ETA: 1s - loss: 0.5890 - accuracy: 0.6947
Epoch 00047: saving model to checkpoints\weights.47.hdf5
282/804 [=========>....................] - ETA: 1s - loss: 0.5867 - accuracy: 0.6982
Epoch 00047: saving model to checkpoints\weights.47.hdf5
386/804 [=============>................] - ETA: 1s - loss: 0.5837 - accuracy: 0.7011
Epoch 00047: saving model to checkpoints\weights.47.hdf5
487/804 [=================>............] - ETA: 0s - loss: 0.5814 - accuracy: 0.7034
Epoch 00047: saving model to checkpoints\weights.47.hdf5
594/804 [=====================>........] - ETA: 0s - loss: 0.5795 - accuracy: 0.7054
Epoch 00047: saving model to checkpoints\weights.47.hdf5
692/804 [========================>.....] - ETA: 0s - loss: 0.5783 - accuracy: 0.7065
Epoch 00047: saving model to checkpoints\weights.47.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5772 - accuracy: 0.7076
Epoch 48/50
  1/804 [..............................] - ETA:

In [57]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_attempt3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5799 - accuracy: 0.7058
Loss: 0.5799351930618286, Accuracy: 0.7057725787162781
